## Packages Import

In [3]:
from flask import Flask
from flask import render_template
from flask import request

import requests
import json
import os

## Flask app 선언

In [4]:
app = Flask(__name__, template_folder=os.getcwd())

## NFT Wallet 사이트 
#### 첫 접속시에는 NFT_Wallet_login.html 파일을 랜더링
#### POST 방식으로 NFT의 스마트컨트랙트 주소를 입력한 경우,
#### > 알맞은 지갑주소일 경우 NFT_Wallet.html 파일 랜더링
#### > 잘못된 지갑주소일 경우 에러 메시지 띄움

In [5]:
@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method=='POST':
        
        contract_address = request.form.to_dict(flat=False)['smart_contract_addr'][0] 
        print(contract_address)
        # 블록 정보 호출
        headers = {'Content-Type' : 'application/json; charset=utf-8'}
        res = requests.get("http://localhost:5000/chain", headers=headers)
        res_json = json.loads(res.content)
        nft_TF = False
        ## 스마트 컨트랙트를 호출 및 실행
        for _block in res_json['chain']:
            for _tx in _block['transactions']:
                if _tx['smart_contract']['contract_address'] == contract_address:
                    exec( _tx['smart_contract']['contract_code']) 
                    nft_TF = True
                    break
        if nft_TF:
#             print(myNFT)
            return render_template("NFT_Wallet.html",  
                                   nft_name = _tx['smart_contract']['contract_code'].split("'")[3], 
                                   nft_img_url = _tx['smart_contract']['contract_code'].split("'")[7],
                                   nft_addresss = contract_address
                                   )
        else:
            return "잘못된 지갑주소입니다."

        
    return render_template('NFT_Wallet_login.html')
app.run(port=8082)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8082/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jan/2023 22:11:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2023 22:17:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2023 22:17:20] "GET /favicon.ico HTTP/1.1" 404 -


13fcfd1807c1bb38ec723d6e3a651d89b62f7a4080b2cd7bcc224a15c0c12ce4


127.0.0.1 - - [22/Jan/2023 22:19:20] "POST / HTTP/1.1" 200 -
